<a href="https://colab.research.google.com/github/CharlesPoletowin/YCBS-273/blob/master/20190813cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# try implement cnn model from pytorch

## import lib

In [0]:
%matplotlib inline
from pathlib import Path
import requests
import gzip
import numpy as np
import math
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

## the original model cnn

In [0]:
class mycnn(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(2, 2), stride=2, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(2, 2), stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.fc_layer=nn.Linear(72,10)

  def forward(self, xb):
    xb = self.conv1(xb)
    xb = self.pool1(xb)
    xb = F.relu(xb)

    xb = self.conv2(xb)
    xb = self.pool2(xb)
    xb = F.relu(xb)

    xb = xb.view(-1,72)  # reshape

    xb = self.fc_layer(xb)
    
    return xb

In [0]:
def train_model(model, x_train, y_train, x_test, y_test, epochs=250, batch_size=64, lr=0.01, weight_decay=0):
  # data
  train_dataset = TensorDataset(x_train, y_train)
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
  
  # loss function
  loss_func = F.cross_entropy

  # optimizer
  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)

  # training loop
  for epoch in range(epochs):
    for xb, yb in train_data_loader:
      
      pred = model(xb)
      loss = loss_func(pred, yb)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    y_pred = model(x_test)
    acc = accuracy_score(torch.argmax(y_pred, dim=1).cpu().detach().numpy(), y_test.cpu())
    
    print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))

## prepare data

In [0]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

def get_data(url, file_name, offset):
  content = requests.get(url).content
  (PATH / file_name).open("wb").write(content)
  with gzip.open((PATH / file_name).as_posix(), "rb") as f:
    data = np.frombuffer(f.read(), dtype=np.uint8, offset=offset)
  return data

y_train = get_data("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", "train-labels", 8).astype('int64')
x_train = get_data("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "train-images", 16).reshape(len(y_train), 784).astype('float32')
y_test = get_data("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", "test-labels", 8).astype('int64')
x_test = get_data("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "test-images", 16).reshape(len(y_test), 784).astype('float32')
x_train, y_train, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_test, y_test)
)
n = x_train.shape[0]


### cuda available

In [0]:
x_test=x_test.cuda()
x_train=x_train.cuda()
y_test=y_test.cuda()
y_train=y_train.cuda()

In [0]:
mycnn1= mycnn()
mycnn1.cuda()

mycnn(
  (conv1): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc_layer): Linear(in_features=72, out_features=10, bias=True)
)

In [0]:
train_model(mycnn1, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.0001)

Loss at epoch 0 : 1.746233, test acc: 0.474800
Loss at epoch 1 : 1.291562, test acc: 0.585600
Loss at epoch 2 : 1.192207, test acc: 0.621900
Loss at epoch 3 : 1.116498, test acc: 0.642300
Loss at epoch 4 : 1.044503, test acc: 0.656400
Loss at epoch 5 : 0.973766, test acc: 0.670800
Loss at epoch 6 : 0.888933, test acc: 0.684900
Loss at epoch 7 : 0.816667, test acc: 0.699000
Loss at epoch 8 : 0.757497, test acc: 0.715800
Loss at epoch 9 : 0.697233, test acc: 0.729500
Loss at epoch 10 : 0.654854, test acc: 0.743300
Loss at epoch 11 : 0.623112, test acc: 0.755800
Loss at epoch 12 : 0.599370, test acc: 0.767900
Loss at epoch 13 : 0.581345, test acc: 0.779400
Loss at epoch 14 : 0.567804, test acc: 0.786000
Loss at epoch 15 : 0.557630, test acc: 0.794200
Loss at epoch 16 : 0.543983, test acc: 0.799900
Loss at epoch 17 : 0.533080, test acc: 0.803600
Loss at epoch 18 : 0.522023, test acc: 0.808000
Loss at epoch 19 : 0.510665, test acc: 0.815300
Loss at epoch 20 : 0.499249, test acc: 0.818600
Lo

In [0]:
train_model(mycnn1, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.0001)

Loss at epoch 0 : 0.383708, test acc: 0.852600
Loss at epoch 1 : 0.375363, test acc: 0.854500
Loss at epoch 2 : 0.365329, test acc: 0.856500
Loss at epoch 3 : 0.357541, test acc: 0.858500
Loss at epoch 4 : 0.346006, test acc: 0.860200
Loss at epoch 5 : 0.334956, test acc: 0.862900
Loss at epoch 6 : 0.325281, test acc: 0.864900
Loss at epoch 7 : 0.314016, test acc: 0.868300
Loss at epoch 8 : 0.304550, test acc: 0.869200
Loss at epoch 9 : 0.295908, test acc: 0.869800
Loss at epoch 10 : 0.289075, test acc: 0.871500
Loss at epoch 11 : 0.282735, test acc: 0.873900
Loss at epoch 12 : 0.279076, test acc: 0.875000
Loss at epoch 13 : 0.272357, test acc: 0.875700
Loss at epoch 14 : 0.267530, test acc: 0.876800
Loss at epoch 15 : 0.261419, test acc: 0.878800
Loss at epoch 16 : 0.258070, test acc: 0.880100
Loss at epoch 17 : 0.254905, test acc: 0.880500
Loss at epoch 18 : 0.252003, test acc: 0.881800
Loss at epoch 19 : 0.249199, test acc: 0.882500
Loss at epoch 20 : 0.245282, test acc: 0.883800
Lo

In [0]:
train_model(mycnn1, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.0001)

Loss at epoch 0 : 0.199632, test acc: 0.890600
Loss at epoch 1 : 0.193213, test acc: 0.891800
Loss at epoch 2 : 0.188237, test acc: 0.893400
Loss at epoch 3 : 0.184487, test acc: 0.894100
Loss at epoch 4 : 0.180569, test acc: 0.894600
Loss at epoch 5 : 0.177181, test acc: 0.895600
Loss at epoch 6 : 0.173735, test acc: 0.896300
Loss at epoch 7 : 0.170623, test acc: 0.896700
Loss at epoch 8 : 0.168641, test acc: 0.897700
Loss at epoch 9 : 0.167001, test acc: 0.898600
Loss at epoch 10 : 0.166416, test acc: 0.899200
Loss at epoch 11 : 0.165784, test acc: 0.899100
Loss at epoch 12 : 0.165009, test acc: 0.899700
Loss at epoch 13 : 0.165316, test acc: 0.900300
Loss at epoch 14 : 0.166102, test acc: 0.900800
Loss at epoch 15 : 0.166592, test acc: 0.901100
Loss at epoch 16 : 0.167382, test acc: 0.902400
Loss at epoch 17 : 0.167557, test acc: 0.902800
Loss at epoch 18 : 0.168183, test acc: 0.903500
Loss at epoch 19 : 0.167581, test acc: 0.904400
Loss at epoch 20 : 0.167333, test acc: 0.904800
Lo

## try new cnn model

In [0]:
class mycnnNew(nn.Module):
    def __init__(self):
      super().__init__()
        
      self.features = nn.Sequential(
          nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(inplace=True),
          nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(32, 64, kernel_size=3, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(inplace=True),
          nn.Conv2d(64, 64, kernel_size=3, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2)
      )
        
      self.classifier = nn.Sequential(
          nn.Dropout(p = 0.5),
          nn.Linear(64 * 7 * 7, 512),
          nn.BatchNorm1d(512),
          nn.ReLU(inplace=True),
          nn.Dropout(p = 0.5),
          nn.Linear(512, 512),
          nn.BatchNorm1d(512),
          nn.ReLU(inplace=True),
          nn.Dropout(p = 0.5),
          nn.Linear(512, 10),
      )
        
      for m in self.features.children():
          if isinstance(m, nn.Conv2d):
              n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
              m.weight.data.normal_(0, math.sqrt(2. / n))
          elif isinstance(m, nn.BatchNorm2d):
              m.weight.data.fill_(1)
              m.bias.data.zero_()
      
      for m in self.classifier.children():
          if isinstance(m, nn.Linear):
              nn.init.xavier_uniform(m.weight)
          elif isinstance(m, nn.BatchNorm1d):
              m.weight.data.fill_(1)
              m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x     

In [0]:
mycnn2= mycnnNew()
mycnn2.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


mycnnNew(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Drop

In [0]:
def train_model(model, x_train, y_train, x_test, y_test, epochs=250, batch_size=256, lr=0.01, weight_decay=0):
  # data
  train_dataset = TensorDataset(x_train, y_train)
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
  
  # loss function
  loss_func = F.cross_entropy

  # optimizer
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

  # training loop
  for epoch in range(epochs):
    for xb, yb in train_data_loader:
      
      pred = model(xb)
      loss = loss_func(pred, yb)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    y_pred = model(x_test)
    acc = accuracy_score(torch.argmax(y_pred, dim=1).cpu().detach().numpy(), y_test.cpu())
    
    print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))

In [0]:
train_model(mycnn2, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 0.024664, test acc: 0.980600


RuntimeError: ignored

## try new cnn model2 without fully connected layer

In [0]:
class mycnnNew2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(2, 2), stride=2, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(2, 2), stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv3 = nn.Conv2d(in_channels=8, out_channels=10, kernel_size=(2, 2), stride=1, padding=0)
    self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

  def forward(self, xb):
    xb = self.conv1(xb)
    xb = self.pool1(xb)
    xb = F.relu(xb)

    xb = self.conv2(xb)
    xb = self.pool2(xb)
    xb = F.relu(xb)

    xb = self.conv3(xb)
    xb = self.pool3(xb)
    xb = F.relu(xb)

    xb = xb.view(-1,10)  # reshape
    
    return xb

In [0]:
mycnn2= mycnnNew2()
mycnn2.cuda()

mycnnNew2(
  (conv1): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(8, 10, kernel_size=(2, 2), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)

In [0]:
train_model(mycnn2, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 1.294848, test acc: 0.657200
Loss at epoch 1 : 0.966768, test acc: 0.752900
Loss at epoch 2 : 0.588402, test acc: 0.796900
Loss at epoch 3 : 0.384869, test acc: 0.821300
Loss at epoch 4 : 0.299969, test acc: 0.827000
Loss at epoch 5 : 0.266450, test acc: 0.832100
Loss at epoch 6 : 0.253402, test acc: 0.835300
Loss at epoch 7 : 0.275465, test acc: 0.839900
Loss at epoch 8 : 0.303268, test acc: 0.842600
Loss at epoch 9 : 0.319065, test acc: 0.845300
Loss at epoch 10 : 0.313331, test acc: 0.848000
Loss at epoch 11 : 0.313610, test acc: 0.850300
Loss at epoch 12 : 0.297781, test acc: 0.852200
Loss at epoch 13 : 0.288426, test acc: 0.853600
Loss at epoch 14 : 0.289039, test acc: 0.853700
Loss at epoch 15 : 0.295336, test acc: 0.855900
Loss at epoch 16 : 0.294172, test acc: 0.856300
Loss at epoch 17 : 0.281977, test acc: 0.857600
Loss at epoch 18 : 0.279066, test acc: 0.856100
Loss at epoch 19 : 0.274937, test acc: 0.854400
Loss at epoch 20 : 0.266525, test acc: 0.853600
Lo

## try cnn new model3 

In [0]:
class mycnnNew3(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(2, 2), stride=2, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 2), stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(2, 2), stride=1, padding=0)
    self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

  def forward(self, xb):
    xb = self.conv1(xb)
    xb = self.pool1(xb)
    xb = F.relu(xb)

    xb = self.conv2(xb)
    xb = self.pool2(xb)
    xb = F.relu(xb)

    xb = self.conv3(xb)
    xb = self.pool3(xb)
    xb = F.relu(xb)

    xb = xb.view(-1,10)  # reshape
    
    return xb

In [0]:
mycnn3= mycnnNew3()
mycnn3.cuda()

mycnnNew3(
  (conv1): Conv2d(1, 4, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(8, 8, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(8, 10, kernel_size=(2, 2), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)

In [0]:
train_model(mycnn3, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 0.841792, test acc: 0.650700
Loss at epoch 1 : 0.503979, test acc: 0.740000
Loss at epoch 2 : 0.276702, test acc: 0.809200
Loss at epoch 3 : 0.314182, test acc: 0.823900
Loss at epoch 4 : 0.286516, test acc: 0.828400
Loss at epoch 5 : 0.246339, test acc: 0.832300
Loss at epoch 6 : 0.231269, test acc: 0.838400
Loss at epoch 7 : 0.223435, test acc: 0.840200
Loss at epoch 8 : 0.221031, test acc: 0.840300
Loss at epoch 9 : 0.226826, test acc: 0.841300
Loss at epoch 10 : 0.216865, test acc: 0.840500
Loss at epoch 11 : 0.205376, test acc: 0.840200
Loss at epoch 12 : 0.185354, test acc: 0.846300
Loss at epoch 13 : 0.168543, test acc: 0.848000
Loss at epoch 14 : 0.159415, test acc: 0.848300
Loss at epoch 15 : 0.154316, test acc: 0.848200
Loss at epoch 16 : 0.150831, test acc: 0.849700
Loss at epoch 17 : 0.159984, test acc: 0.852600
Loss at epoch 18 : 0.154055, test acc: 0.853500
Loss at epoch 19 : 0.151885, test acc: 0.852900
Loss at epoch 20 : 0.154190, test acc: 0.853700
Lo

## try new cnn model4 improve avgpool2d

In [0]:
class mycnnNew4(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(2, 2), stride=2, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 2), stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(2, 2), stride=1, padding=0)
    self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), padding=0)

  def forward(self, xb):
    xb = self.conv1(xb)
    xb = self.pool1(xb)
    xb = F.leaky_relu(xb)

    xb = self.conv2(xb)
    xb = self.pool2(xb)
    xb = F.leaky_relu(xb)

    xb = self.conv3(xb)
    xb = self.pool3(xb)
    xb = F.leaky_relu(xb)

    xb = xb.view(-1,10)  # reshape
    
    return xb

In [0]:
mycnn4= mycnnNew4()
mycnn4.cuda()

mycnnNew4(
  (conv1): Conv2d(1, 8, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 10, kernel_size=(2, 2), stride=(1, 1))
  (pool3): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
)

In [0]:
train_model(mycnn4, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 2.079775, test acc: 0.289800
Loss at epoch 1 : 1.795111, test acc: 0.393100
Loss at epoch 2 : 1.538747, test acc: 0.531300
Loss at epoch 3 : 1.232358, test acc: 0.663300
Loss at epoch 4 : 0.631396, test acc: 0.821700
Loss at epoch 5 : 0.554915, test acc: 0.852800
Loss at epoch 6 : 0.466510, test acc: 0.871800
Loss at epoch 7 : 0.407404, test acc: 0.884600
Loss at epoch 8 : 0.372852, test acc: 0.894700
Loss at epoch 9 : 0.335485, test acc: 0.902800
Loss at epoch 10 : 0.316834, test acc: 0.909700
Loss at epoch 11 : 0.303526, test acc: 0.914100
Loss at epoch 12 : 0.288920, test acc: 0.917000
Loss at epoch 13 : 0.279811, test acc: 0.920100
Loss at epoch 14 : 0.268737, test acc: 0.924100
Loss at epoch 15 : 0.259610, test acc: 0.925100
Loss at epoch 16 : 0.249873, test acc: 0.927800
Loss at epoch 17 : 0.243584, test acc: 0.930400
Loss at epoch 18 : 0.238489, test acc: 0.931200
Loss at epoch 19 : 0.233559, test acc: 0.932400
Loss at epoch 20 : 0.227727, test acc: 0.933300
Lo

## try cnn new model5

In [0]:
class mycnnNew5(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(2, 2), stride=2, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv2 = nn.Conv2d(in_channels=2, out_channels=5, kernel_size=(2, 2), stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), padding=0)

    self.conv3 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=(2, 2), stride=1, padding=1)
    self.pool3 = nn.AvgPool2d(kernel_size=(2, 2), padding=0)

  def forward(self, xb):
    xb = self.conv1(xb)
    xb = self.pool1(xb)
    xb = F.leaky_relu(xb)

    xb = self.conv2(xb)
    xb = self.pool2(xb)
    xb = F.leaky_relu(xb)

    xb = self.conv3(xb)
    xb = self.pool3(xb)
    xb = F.leaky_relu(xb)

    xb = xb.view(-1,10)  # reshape
    
    return xb

In [0]:
mycnn5 = mycnnNew5()
mycnn5.cuda()

mycnnNew5(
  (conv1): Conv2d(1, 2, kernel_size=(2, 2), stride=(2, 2))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(2, 5, kernel_size=(2, 2), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(5, 10, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (pool3): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
)

In [0]:
train_model(mycnn5, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

ValueError: ignored

## summarize

In [0]:
list(mycnnNew.parameters(mycnn2))

[Parameter containing:
 tensor([[[[ 0.0769, -0.1252],
           [ 0.1644, -0.0865]]],
 
 
         [[[ 0.0918,  0.0525],
           [-0.0318,  0.0551]]],
 
 
         [[[-0.0019, -0.1170],
           [-0.1328, -0.2068]]],
 
 
         [[[-0.0667, -0.1217],
           [ 0.0988,  0.0707]]]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.0052, -0.0009, -0.1874, -0.0051], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([[[[-0.0599, -0.0715],
           [-0.0180, -0.0408]],
 
          [[-0.0755, -0.1858],
           [ 0.0995,  0.1057]],
 
          [[-0.0839, -0.0528],
           [-0.1902, -0.0899]],
 
          [[ 0.2481,  0.2178],
           [ 0.0613,  0.0429]]],
 
 
         [[[ 0.0442, -0.0330],
           [-0.0565, -0.0597]],
 
          [[ 0.0829, -0.0599],
           [ 0.2859,  0.2950]],
 
          [[ 0.0983,  0.1306],
           [-0.1690,  0.1080]],
 
          [[-0.0123,  0.0940],
           [ 0.0414,  0.1103]]],
 
 
      